# Module 8: 9 - Anatomy of an AI Agent - From Tool to Function Call Format with Pydantic
---------------------------------------------------------------------------------
In this lesson, we will streamline the process of converting Python function signatures to OpenAI Function Calling format using Pydantic models. By leveraging the [model_json_schema()](https://docs.pydantic.dev/latest/concepts/json_schema/) method provided by Pydantic, we can generate JSON schemas compliant with JSON Schema Draft 2020-12 and OpenAPI Specification v3.1.0. This approach simplifies the creation of function signatures for OpenAI's Function Calling, enabling seamless tool execution. We will then proceed with the execution process using these validated and formatted tool arguments.

## Objectives
* Simplify the conversion of Python function signatures to OpenAI Function Calling format using Pydantic.
* Understand how to generate JSON schemas from Pydantic models.
* Integrate these JSON schemas into the tool execution process.
* Validate and execute tools with the AI Agent using the generated schemas.

## What this session covers:
* Defining the current agent structure, including the LLM Client and short-term memory.
* Creating Pydantic models for tool argument validation.
* Generating JSON schemas from Pydantic models using model_json_schema().
* Converting these JSON schemas to OpenAI Function Calling format.
* Integrating the schemas into the AI Agent for tool execution.
* Initializing and testing the AI Agent with simplified tool execution.

## Install Libraries

In [2]:
#! pip install openai

## Define Current Agent Structure

### LLM Client

In [1]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """Interacts with OpenAI's API for chat completions."""
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[str], tools: List[Dict[str, Any]] = None, **kwargs) -> Dict[str, Any]:
        """Generates a response from OpenAI's API."""
        params = {'messages': messages, 'model': self.model, 'tools': tools, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

### Short-Term Memory

In [2]:
from typing import List, Dict

class ChatMessageMemory:
    """Manages conversation context."""
    
    def __init__(self):
        self.messages = []
    
    def add_message(self, message: Dict):
        """Add a message to memory."""
        self.messages.append(message)
    
    def add_messages(self, messages: List[Dict]):
        """Add multiple messages to memory."""
        for message in messages:
            self.add_message(message)
    
    def get_messages(self) -> List[Dict]:
        """Retrieve all messages."""
        return self.messages.copy()
    
    def reset_memory(self):
        """Clear all messages."""
        self.messages = []

### Agent Base

In [3]:
from typing import Dict

class Agent:
    """Integrates LLM client, tools, and memory."""
    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history, tools=self.tools)
        self.memory.add_message(response)
        return response

## Defining Dummy Tools

In [7]:
import random

def get_weather(location: str) -> str:
    """Gets weather information."""
    temperature = random.randint(60, 80)
    return f"{location}: {temperature}F."

def jump(distance: str) -> str:
    """Jumps a specific distance."""
    return f"I jumped the following distance {distance}"

## Define Tool Schemas

In [8]:
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    """Get weather information based on location."""
    location: str = Field(description="Location to get weather for")

class JumpSchema(BaseModel):
    """Jump a specific distance"""
    distance: str = Field(description="Specific distance to jump for")

## Convert Schemas to OpenAI Function Calling Format

### Current Approach

In [9]:
get_weather_func_dict = {
    'type': 'function',
    'function': {
        'name': 'get_weather',
        'description': 'Get weather information based on location.',
        'parameters': {
            'properties': {'location': {'type': 'string'}},
            'required': ['location'],
            'type': 'object'
        }
    }
}

jump_func_dict = {
    'type': 'function',
    'function': {
        'name': 'jump',
        'description': 'Jump a specific distance.',
        'parameters': {
            'properties': {'distance': {'type': 'string'}},
            'required': ['distance'],
            'type': 'object'
        }
    }
}

tools = [get_weather_func_dict, jump_func_dict]

### Pydantic Approach

In [12]:
from pydantic import BaseModel

def to_openai_function_call_definition(name: str, model: BaseModel):
    schema_dict = model.model_json_schema()
    description = schema_dict.pop("description", "No description provided")
    schema_dict.pop("title", None)  # Remove the title field to exclude the model name
    return {
        "type": "function",
        "function": {
            "name": name,
            "description": description,
            "parameters": schema_dict
        }
    }

get_weather_func_dict = to_openai_function_call_definition("get_weather", GetWeatherSchema)
jump_func_dict = to_openai_function_call_definition("jump", JumpSchema)

tools = [get_weather_func_dict, jump_func_dict]

In [13]:
"""
get_weather_func_dict = {
    'type': 'function',
    'function': {
        'name': 'get_weather',
        'description': 'Get weather information based on location.',
        'parameters': {
            'properties': {'location': {'type': 'string'}},
            'required': ['location'],
            'type': 'object'
        }
    }
}
"""
tools[0]

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get weather information based on location.',
  'parameters': {'properties': {'location': {'description': 'Location to get weather for',
     'title': 'Location',
     'type': 'string'}},
   'required': ['location'],
   'type': 'object'}}}

## Initialize Client

In [14]:
# API from environment variable
# import os
# api_key = os.getenv("OPENAI_API_KEY"))

api_key=""

client = OpenAIChatCompletion(
    base_url='https://api.openai.com/v1',
    model='gpt-4',
    api_key=api_key
)

## Define System messages

In [15]:
# Define the system message
system_message = {"role": "system", "content": "You are a weather assistant."}

## Initialize Agent

In [16]:
# Initialize the Agent with the LLM client, system message, and tools
agent = Agent(llm_client=client, system_message=system_message, tools=tools)

## Send a User Message

In [17]:
# Define a user message
user_message = {"role": "user", "content": "What is the weather in Virginia?"}

# Generate a response using the agent
response = agent.run(user_message)
response

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_140RrQ72jVF3kMjQTiqgYpih', function=Function(arguments='{\n  "location": "Virginia"\n}', name='get_weather'), type='function')])

In [18]:
response.to_dict()

{'content': None,
 'role': 'assistant',
 'tool_calls': [{'id': 'call_140RrQ72jVF3kMjQTiqgYpih',
   'function': {'arguments': '{\n  "location": "Virginia"\n}',
    'name': 'get_weather'},
   'type': 'function'}]}

## Parse Tool Response

In [19]:
tool_response = response.tool_calls[0]
tool_arguments_json = tool_response.function.arguments
tool_arguments_json

'{\n  "location": "Virginia"\n}'

## Validate Function Signature

### Validate Tool Arguments - JSON Format

In [20]:
response_model = GetWeatherSchema.model_validate_json(tool_arguments_json)
response_model

GetWeatherSchema(location='Virginia')

### Convert to Dictionary

In [21]:
tool_arguments_dict = response_model.model_dump()
tool_arguments_dict

{'location': 'Virginia'}

## Execute Tool

In [22]:
get_weather(**tool_arguments_dict)

'Virginia: 75F.'